**Import necessary libraries**

In [1]:
pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [2]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.7 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=73be4c2ee08945fffac449f8f6ef66671e0cbe8950d1e75b9543008542cee1b2
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8d5737e2592ef9ae028405b78e8e679933aa5512a642ccf0cabf508ed1ec7e7e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [3]:
import re
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [4]:
import nltk

# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import gradio as gr
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration

In [15]:
!pip install -U gradio langchain langchain-community pypdf langchain-google-genai langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [17]:
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
import gradio as gr

**Insert API Key**

In [16]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

**Load Models**

In [6]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

**Frequency-based extractive summarization method**

In [7]:
def frequency_method(text, max_sentences=3):
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())

    # Remove punctuation and calculate word frequency
    words = [word for word in words if word.isalnum()]
    word_freq = Counter(words)

    # Score sentences based on word frequency
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_freq:
                sentence_scores[sentence] = sentence_scores.get(sentence, 0) + word_freq[word]

    # Sort sentences by score and return the top sentences
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:max_sentences]
    summary = ' '.join(summary_sentences)
    return summary

**Sumy-based extractive summarization method**

In [8]:
def sumy_method(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 2)
    return " ".join(str(sentence) for sentence in summary)

**Abstractive summarization using T5 model**

In [9]:
def summarize_with_t5(text, max_words):
    inputs = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=max_words, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

**Abstractive summarization using BART model**

In [10]:
def summarize_with_bart(text, max_words):
    inputs = bart_tokenizer.encode("summarize in f'{max_words}' words: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=max_words, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

**Logic handling function**

In [11]:
def summarize(text, summarization_type, method):
    if summarization_type == "Extractive":
        if method == "Frequency Method":
            return frequency_method(text)
        elif method == "Sumy Library":
            return sumy_method(text)
    elif summarization_type == "Abstractive":
        if method == "T5 Model":
            return summarize_with_t5(text)
        elif method == "BART Model":
            return summarize_with_bart(text)

In [12]:
def summarize_text(text, algorithm, method, max_words=50):
    word_count = len(text.split())
    if max_words < 30:
        max_words = 30

    if algorithm == "Extractive":
        summary = frequency_method(text, max_words) if method == "Frequency Method" else sumy_method(text, max_words)
    else:
        summary = summarize_with_t5(text, max_words) if method == "T5" else summarize_with_bart(text, max_words)

    summary_word_count = len(summary.split())
    return summary, word_count, summary_word_count

In [13]:
def update_algorithm(summarization_type):
    if summarization_type == "Extractive":
        return gr.Dropdown.update(choices=["Frequency Method", "Sumy Library"], value="Frequency Method")
    elif summarization_type == "Abstractive":
        return gr.Dropdown.update(choices=["T5", "BART"], value="T5")

In [14]:
# Main function to process user selections
def generate_summary(text, summarization_type, algorithm, max_words=50):
    # Get word count of input text
    input_word_count = len(text.split())

    # Default minimum for summary length
    max_words = max(30, max_words)

    if summarization_type == "Extractive":
        if algorithm == "Frequency Method":
            summary = frequency_method(text)
        elif algorithm == "Sumy Library":
            summary = sumy_method(text)
    elif summarization_type == "Abstractive":
        if algorithm == "T5":
            summary = summarize_with_t5(text, max_words)
        elif algorithm == "BART":
            summary = summarize_with_bart(text, max_words)

    # Get word count of generated summary
    summary_word_count = len(summary.split())

    return f"{summary}\n\nInput Word Count: {input_word_count}\nSummary Word Count: {summary_word_count}"

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

# LLM Initialization
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# LLM Summarization Function
def llm_summarization(text, max_words=50):
    # Generate a summary based on input text using LLM
    return f"LLM-based summary for {len(text.split())} words with word limit {max_words}."

# PDF Summarization Function
def summarize_pdf(pdf_file):
    loader = PyPDFLoader(pdf_file.name)
    docs = loader.load_and_split()
    chain = load_summarize_chain(llm, chain_type="refine")
    summary = chain.invoke(docs)
    return summary['output_text']

**Gradio interface**

In [20]:
import gradio as gr

# Generate summary based on user input or uploaded file
def generate_summary_v2(text, pdf_file, summarization_type, algorithm, max_words=50):
    # Ensure max_words is at least 30
    max_words = max(30, max_words)

    # Determine input source: Textbox or PDF file
    if pdf_file:
        summary = summarize_pdf(pdf_file)
        input_word_count = len(summary.split())  # Use PDF word count
    else:
        input_word_count = len(text.split())

        if summarization_type == "Extractive":
            if algorithm == "Frequency Method":
                summary = frequency_method(text)
            elif algorithm == "Sumy Library":
                summary = sumy_method(text)
        elif summarization_type == "Abstractive":
            if algorithm == "T5":
                summary = summarize_with_t5(text, max_words)
            elif algorithm == "BART":
                summary = summarize_with_bart(text, max_words)
        elif summarization_type == "LLM":
            summary = llm_summarization(text, max_words)
        else:
            summary = "Invalid summarization type selected."

    # Get word count of generated summary
    summary_word_count = len(summary.split())

    return f"{summary}\n\nInput Word Count: {input_word_count}\nSummary Word Count: {summary_word_count}"

# Update algorithm dropdown based on summarization type
def update_algorithm_v2(summarization_type):
    if summarization_type == "Extractive":
        return gr.Dropdown.update(choices=["Frequency Method", "Sumy Library"], value="Frequency Method")
    elif summarization_type == "Abstractive":
        return gr.Dropdown.update(choices=["T5", "BART"], value="T5")
    elif summarization_type == "LLM":
        return gr.Dropdown.update(choices=["LLM Algorithm"], value="LLM Algorithm")

# Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("## Text Summarizer with File Upload Option")

    with gr.Row():
        text_input = gr.Textbox(lines=10, placeholder="Enter the text to summarize", label="Input Text", interactive=True)
        pdf_file = gr.File(label="Upload PDF (Optional)", file_types=[".pdf"])

    with gr.Row():
        summarization_type = gr.Dropdown(choices=["Extractive", "Abstractive", "LLM"], label="Summarization Type")
        algorithm = gr.Dropdown(choices=["Frequency Method", "Sumy Library", "T5", "BART"], label="Algorithm")

    max_words = gr.Number(value=50, label="Maximum Word Count for Summary")

    output = gr.Textbox(label="Generated Summary", lines=10, interactive=False)

    summarization_type.change(update_algorithm_v2, inputs=summarization_type, outputs=algorithm)

    submit_button = gr.Button("Generate Summary")
    submit_button.click(
        generate_summary_v2,
        inputs=[text_input, pdf_file, summarization_type, algorithm, max_words],
        outputs=output
    )

# Launch interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7428e0a581cd2daa29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
